## Notebook Params

In [2]:
re_fetch_flag = False
update_chroma_db = False

## Helper Functions

In [3]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

## Setup

In [4]:
!pip install -U -q langchain langchain-chroma langchain_huggingface langchain_community sentence_transformers openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:

In [5]:
# !pip install openai  chromadb faiss-cpu pypdf tiktoken docarray --quiet

In [6]:
# Import the necessary libraies
import os
import base64
import requests
from collections import Counter
import pickle
import openai

from langchain_core.documents import Document
from langchain.vectorstores import Chroma

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [8]:
# Set the API key by reading the folder path. Use this code if you're running the code on Google Colab. Otherwise, use the actual folder path
folder_path = '/content/drive/MyDrive/upgrad'

# Folder path
os.chdir(folder_path)

## Set the OpenAI & GitHub Keys

In [9]:
# Read the text file containing the API key
with open("api_keys/OpenAI_key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

# Update the OpenAI API key by updating the environment variable
os.environ["OPENAI_API_KEY"] = openai.api_key

# Read the text file containing the API key
with open("api_keys/Github_key.txt", "r") as f:
  github_key = ' '.join(f.readlines())

# Update the OpenAI API key by updating the environment variable
os.environ['GITHUB_PERSONAL_ACCESS_TOKEN'] = github_key

## Getting Files from Github

In [10]:
headers = {
  "Accept": "application/vnd.github+json",
  "Authorization": f"Bearer {os.environ['GITHUB_PERSONAL_ACCESS_TOKEN']}",
}

base_url = (
  f"https://api.github.com/repos/dmlc/xgboost/contents"
)

def get_content(files) :
  return [requests.get(x['url'], headers=headers).json() for x in files]

In [11]:
response = requests.get(base_url, headers=headers)
response.raise_for_status()
files = response.json()
menu = get_content(files)

In [12]:
final_menu = []

def flat_content(menu=menu, debug=False):

  for item in menu:

    if isinstance(item,list) :
      menu = get_content(item)
      flat_content(menu)

    else :
      if debug :
        print(item['path'])
      final_menu.append(item)

In [13]:
if re_fetch_flag:
    flat_content(menu=menu, debug=True)

In [14]:
if re_fetch_flag:

    with open("Course 5 - SemanticSpotter /Final_Project/all_files.pkl", "wb") as file:
        pickle.dump(final_menu, file)
else:
    with open("Course 5 - SemanticSpotter /Final_Project/all_files.pkl", "rb") as file:
        final_menu = pickle.load(file)

In [15]:
final_menu[0], final_menu[-1]

({'name': '.clang-format',
  'path': '.clang-format',
  'sha': '737cf9006baef17d7aa40da120587aeb641e0692',
  'size': 5763,
  'url': 'https://api.github.com/repos/dmlc/xgboost/contents/.clang-format?ref=master',
  'html_url': 'https://github.com/dmlc/xgboost/blob/master/.clang-format',
  'git_url': 'https://api.github.com/repos/dmlc/xgboost/git/blobs/737cf9006baef17d7aa40da120587aeb641e0692',
  'download_url': 'https://raw.githubusercontent.com/dmlc/xgboost/master/.clang-format',
  'type': 'file',
  'content': 'LS0tCkxhbmd1YWdlOiAgICAgICAgQ3BwCiMgQmFzZWRPblN0eWxlOiAgR29v\nZ2xlCkFjY2Vzc01vZGlmaWVyT2Zmc2V0OiAtMQpBbGlnbkFmdGVyT3BlbkJy\nYWNrZXQ6IEFsaWduCkFsaWduQXJyYXlPZlN0cnVjdHVyZXM6IE5vbmUKQWxp\nZ25Db25zZWN1dGl2ZU1hY3JvczogTm9uZQpBbGlnbkNvbnNlY3V0aXZlQXNz\naWdubWVudHM6IE5vbmUKQWxpZ25Db25zZWN1dGl2ZUJpdEZpZWxkczogTm9u\nZQpBbGlnbkNvbnNlY3V0aXZlRGVjbGFyYXRpb25zOiBOb25lCkFsaWduRXNj\nYXBlZE5ld2xpbmVzOiBMZWZ0CkFsaWduT3BlcmFuZHM6ICAgQWxpZ24KQWxp\nZ25UcmFpbGluZ0NvbW1lbnRzOiB0cnVlCkFsbG93QWxsQXJndW

In [16]:
final_menu_copy = final_menu.copy()

### Exploring Files

In [17]:
Counter([1 if 'content' in x else 0  for x in final_menu])

Counter({1: 1339, 0: 2})

In [18]:
len(final_menu)

1341

In [19]:
[x for x in final_menu if 'content' not in x][0]

{'name': 'dmlc-core',
 'path': 'dmlc-core',
 'sha': '13341857549852a9a86b1894b5ba84c6276ab381',
 'size': 0,
 'url': 'https://api.github.com/repos/dmlc/xgboost/contents/dmlc-core?ref=master',
 'html_url': 'https://github.com/dmlc/dmlc-core/tree/13341857549852a9a86b1894b5ba84c6276ab381',
 'git_url': 'https://api.github.com/repos/dmlc/dmlc-core/git/trees/13341857549852a9a86b1894b5ba84c6276ab381',
 'download_url': None,
 'type': 'submodule',
 'submodule_git_url': 'https://github.com/dmlc/dmlc-core',
 '_links': {'self': 'https://api.github.com/repos/dmlc/xgboost/contents/dmlc-core?ref=master',
  'git': 'https://api.github.com/repos/dmlc/dmlc-core/git/trees/13341857549852a9a86b1894b5ba84c6276ab381',
  'html': 'https://github.com/dmlc/dmlc-core/tree/13341857549852a9a86b1894b5ba84c6276ab381'}}

## Filtering Final Menu

In [20]:
final_menu = [x for x in final_menu if 'content' in x]

In [21]:
final_menu = [
    x for x in final_menu if (
                              (
                                ('.rst' in x['path'])|
                                ('doc/' in x['path'])|
                                ('demo/' in x['path'])|
                                ('tutorials/' in x['path'])
                              ) &
                              (
                                ('data' not in x['path']) |
                                ('data' not in x['name'])
                              )
                             )
]

In [22]:
len(final_menu)

221

In [23]:
print(*[x['path'] for x in final_menu],sep=", ")

R-package/demo/00Index, R-package/demo/README.md, R-package/demo/basic_walkthrough.R, R-package/demo/boost_from_prediction.R, R-package/demo/create_sparse_matrix.R, R-package/demo/cross_validation.R, R-package/demo/custom_objective.R, R-package/demo/early_stopping.R, R-package/demo/generalized_linear_model.R, R-package/demo/gpu_accelerated.R, R-package/demo/interaction_constraints.R, R-package/demo/poisson_regression.R, R-package/demo/predict_first_ntree.R, R-package/demo/predict_leaf_indices.R, R-package/demo/runall.R, R-package/demo/tweedie_regression.R, demo/.gitignore, demo/CLI/README.rst, demo/CLI/binary_classification/README.md, demo/CLI/binary_classification/agaricus-lepiota.fmap, demo/CLI/binary_classification/agaricus-lepiota.names, demo/CLI/binary_classification/mapfeat.py, demo/CLI/binary_classification/mknfold.py, demo/CLI/binary_classification/mushroom.conf, demo/CLI/binary_classification/runexp.sh, demo/CLI/distributed-training/README.md, demo/CLI/distributed-training/mus

In [24]:
Counter([x['type'] for x in final_menu ])

Counter({'file': 221})

In [25]:
final_menu[0]

{'name': '00Index',
 'path': 'R-package/demo/00Index',
 'sha': 'fa09fa900486de72148e229d75afbdd4ba40768b',
 'size': 809,
 'url': 'https://api.github.com/repos/dmlc/xgboost/contents/R-package/demo/00Index?ref=master',
 'html_url': 'https://github.com/dmlc/xgboost/blob/master/R-package/demo/00Index',
 'git_url': 'https://api.github.com/repos/dmlc/xgboost/git/blobs/fa09fa900486de72148e229d75afbdd4ba40768b',
 'download_url': 'https://raw.githubusercontent.com/dmlc/xgboost/master/R-package/demo/00Index',
 'type': 'file',
 'content': 'YmFzaWNfd2Fsa3Rocm91Z2ggICAgICAgICAgICAgICBCYXNpYyBmZWF0dXJl\nIHdhbGt0aHJvdWdoCmN1c3RvbV9vYmplY3RpdmUgICAgICAgICAgICAgICAg\nQ3VzdG9taXplIGxvc3MgZnVuY3Rpb24sIGFuZCBldmFsdWF0aW9uIG1ldHJp\nYwpib29zdF9mcm9tX3ByZWRpY3Rpb24gICAgICAgICAgIEJvb3N0aW5nIGZy\nb20gZXhpc3RpbmcgcHJlZGljdGlvbgpwcmVkaWN0X2ZpcnN0X250cmVlICAg\nICAgICAgICAgIFByZWRpY3RpbmcgdXNpbmcgZmlyc3QgbiB0cmVlcwpnZW5l\ncmFsaXplZF9saW5lYXJfbW9kZWwgICAgICAgIEdlbmVyYWxpemVkIExpbmVh\nciBNb2RlbApjcm9zc192YWxpZGF0aW9

## Chunking of Documents

In [26]:
def get_file_content(content_encoded) :
  return base64.b64decode(content_encoded).decode("utf-8")

import re

def parse_content(text, debug=False):

    # Regular expressions for the headings
    heading_patterns = [
        r"(?P<heading0>^[A-Za-z0-9 \-]+)\n(?P<underline0>\#+)$",
        r"(?P<heading1>^[A-Za-z0-9 \-]+)\n(?P<underline1>=+)$",
        r"(?P<heading2A>^[A-Za-z0-9 \-]+)\n(?P<underline2>-+)$",
        r"(?P<heading2B>^[A-Za-z0-9 \-]+)\n(?P<underline3>\*+)$",
        r"(?P<heading3>\* ``[A-Za-z0-9_\-]+``)" #experimental
    ]

    # Combine all patterns into one
    combined_pattern = "|".join(heading_patterns)


    matches = re.finditer(combined_pattern, text, re.MULTILINE)

    headings = []
    last_index = 0

    for match in matches:
        if debug :
            print(match)

        start, end = match.span()

        if last_index < start:
            content = text[last_index:start].strip()
            if all([content, headings]):
                headings[-1]['content'].append(content)

        if match.group('heading0'):
            headings.append({'heading': match.group('heading0').strip(), 'level': 0, 'content': []})
        elif match.group('heading1'):
            headings.append({'heading': match.group('heading1').strip(), 'level': 1, 'content': []})
        elif match.group('heading2A'):
            headings.append({'heading': match.group('heading2A').strip(), 'level': 2, 'content': []})
        elif match.group('heading2B'):
            headings.append({'heading': match.group('heading2B').strip(), 'level': 2, 'content': []})
        elif match.group('heading3'):
            headings.append({'heading': match.group('heading3').strip(), 'level': 3, 'content': []})

        last_index = end

    # Capture the remaining content after the last match
    if last_index < len(text):
        content = text[last_index:].strip()
        if all([content, headings]):
            headings[-1]['content'].append(content)
        else :
            headings.append({'heading':'' , 'level': 0, 'content': content})


    return headings

In [27]:
param_page = [x for x in final_menu if x['path'] == 'doc/parameter.rst'][0]
param_page

{'name': 'parameter.rst',
 'path': 'doc/parameter.rst',
 'sha': 'a776559223f4ab9f1684e872bbf9bd335389d821',
 'size': 34032,
 'url': 'https://api.github.com/repos/dmlc/xgboost/contents/doc/parameter.rst?ref=master',
 'html_url': 'https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst',
 'git_url': 'https://api.github.com/repos/dmlc/xgboost/git/blobs/a776559223f4ab9f1684e872bbf9bd335389d821',
 'download_url': 'https://raw.githubusercontent.com/dmlc/xgboost/master/doc/parameter.rst',
 'type': 'file',
 'content': '...\ncHJvYmFiaWxpdHkK\n',
 'encoding': 'base64',
 '_links': {'self': 'https://api.github.com/repos/dmlc/xgboost/contents/doc/parameter.rst?ref=master',
  'git': 'https://api.github.com/repos/dmlc/xgboost/git/blobs/a776559223f4ab9f1684e872bbf9bd335389d821',
  'html': 'https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst'}}

In [28]:
# print(get_file_content(param_page['content']))

In [29]:
[(x['heading'],x['level']) for x in parse_content(get_file_content(param_page['content']))]

[('XGBoost Parameters', 0),
 ('Global Configuration', 2),
 ('* ``verbosity``', 3),
 ('* ``use_rmm``', 3),
 ('General Parameters', 2),
 ('* ``booster``', 3),
 ('* ``device``', 3),
 ('* ``verbosity``', 3),
 ('* ``validate_parameters``', 3),
 ('* ``nthread``', 3),
 ('* ``disable_default_eval_metric``', 3),
 ('Parameters for Tree Booster', 1),
 ('* ``eta``', 3),
 ('* ``gamma``', 3),
 ('* ``max_depth``', 3),
 ('* ``min_child_weight``', 3),
 ('* ``max_delta_step``', 3),
 ('* ``subsample``', 3),
 ('* ``sampling_method``', 3),
 ('* ``colsample_bytree``', 3),
 ('* ``lambda``', 3),
 ('* ``alpha``', 3),
 ('* ``tree_method``', 3),
 ('* ``scale_pos_weight``', 3),
 ('* ``updater``', 3),
 ('* ``refresh_leaf``', 3),
 ('* ``process_type``', 3),
 ('* ``grow_policy``', 3),
 ('* ``max_leaves``', 3),
 ('* ``max_bin``', 3),
 ('* ``num_parallel_tree``', 3),
 ('* ``monotone_constraints``', 3),
 ('* ``interaction_constraints``', 3),
 ('* ``multi_strategy``', 3),
 ('* ``max_cached_hist_node``', 3),
 ('Parameter

In [30]:
get_file_content(param_page['content'])

'##################\nXGBoost Parameters\n##################\nBefore running XGBoost, we must set three types of parameters: general parameters, booster parameters and task parameters.\n\n- **General parameters** relate to which booster we are using to do boosting, commonly tree or linear model\n- **Booster parameters** depend on which booster you have chosen\n- **Learning task parameters** decide on the learning scenario. For example, regression tasks may use different parameters with ranking tasks.\n- **Command line parameters** relate to behavior of CLI version of XGBoost.\n\n.. note:: Parameters in R package\n\n  In R-package, you can use ``.`` (dot) to replace underscore in the parameters, for example, you can use ``max.depth`` to indicate ``max_depth``. The underscore parameters are also valid in R.\n\n.. contents::\n  :backlinks: none\n  :local:\n\n\n.. _global_config:\n\n********************\nGlobal Configuration\n********************\nThe following parameters can be set in the 

In [31]:
parse_content(get_file_content(param_page['content']))

[{'heading': 'XGBoost Parameters',
  'level': 0,
  'content': ['Before running XGBoost, we must set three types of parameters: general parameters, booster parameters and task parameters.\n\n- **General parameters** relate to which booster we are using to do boosting, commonly tree or linear model\n- **Booster parameters** depend on which booster you have chosen\n- **Learning task parameters** decide on the learning scenario. For example, regression tasks may use different parameters with ranking tasks.\n- **Command line parameters** relate to behavior of CLI version of XGBoost.\n\n.. note:: Parameters in R package\n\n  In R-package, you can use ``.`` (dot) to replace underscore in the parameters, for example, you can use ``max.depth`` to indicate ``max_depth``. The underscore parameters are also valid in R.\n\n.. contents::\n  :backlinks: none\n  :local:\n\n\n.. _global_config:\n\n********************']},
 {'heading': 'Global Configuration',
  'level': 2,
  'content': ['The following p

In [32]:
def lazy_load(files):
  for file in files:
      content = get_file_content(file["content"])
      parsed_content = parse_content(content)

      metadata = {
          "name"    : file["name"],
          "path"    : file["path"],
          "source"  : file["html_url"],
          "doc_api" : file["url"]
      }

      prevHeading0, prevHeading1, prevHeading2, prevHeading3 = "", "", "", ""
      prevContent0, prevContent1, prevContent2, prevContent3 = "", "", "", ""

      for doc_dict in parsed_content :

        try :

          final_heading = ""

          if doc_dict['level'] == 0:
            prevHeading0 = doc_dict['heading']
            prevContent0 = doc_dict['content']
            final_heading = doc_dict['heading']
            final_content = doc_dict['content']

          elif doc_dict['level'] == 1:
            prevHeading1 = doc_dict['heading']
            prevContent1 = doc_dict['content']
            final_heading = f"{prevHeading0} - {doc_dict['heading']}"
            final_content = doc_dict['content']

          elif doc_dict['level'] == 2:
            prevHeading2 = doc_dict['heading']
            prevContent2 = doc_dict['content']
            final_heading = f"{prevHeading0} - {prevHeading1} - {doc_dict['heading']}"
            final_content = doc_dict['content']

          elif doc_dict['level'] == 3:
            prevHeading3 = doc_dict['heading']
            final_heading = f"{prevHeading0} - {prevHeading1} - {prevHeading2}"
            final_content = str(prevContent0) + "\n" + str(doc_dict['heading']) + ":" + str(doc_dict['content'])

          if isinstance(final_content, str)==False:
            final_content = str(final_content)

          final_content = final_heading.upper() + "\n\n" + final_content

          yield Document(page_content=final_content, metadata=metadata)

        except Exception as e :
          print(e)

          yield Document(page_content="404 Error", metadata=metadata)


def load(files):
  return [x for x in lazy_load(files)]

In [33]:
# final_menu_docs = lazy_load(final_menu)
Documents = load(final_menu)

**Count of Chunks**

In [34]:
len(Documents)

681

In [35]:
len([x for x in Documents if x.metadata['path'] == 'doc/parameter.rst'])

81

In [36]:
for chunk in [x for x in Documents if x.metadata['path'] == 'doc/parameter.rst'][0:3] :
  print(chunk.metadata)
  print(chunk.page_content)

  print("\n\n\n","-"*50)

{'name': 'parameter.rst', 'path': 'doc/parameter.rst', 'source': 'https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst', 'doc_api': 'https://api.github.com/repos/dmlc/xgboost/contents/doc/parameter.rst?ref=master'}
XGBOOST PARAMETERS

['Before running XGBoost, we must set three types of parameters: general parameters, booster parameters and task parameters.\n\n- **General parameters** relate to which booster we are using to do boosting, commonly tree or linear model\n- **Booster parameters** depend on which booster you have chosen\n- **Learning task parameters** decide on the learning scenario. For example, regression tasks may use different parameters with ranking tasks.\n- **Command line parameters** relate to behavior of CLI version of XGBoost.\n\n.. note:: Parameters in R package\n\n  In R-package, you can use ``.`` (dot) to replace underscore in the parameters, for example, you can use ``max.depth`` to indicate ``max_depth``. The underscore parameters are also valid in R.

## Improving Document Quality

In [37]:
Documents[0].page_content

'\n\nbasic_walkthrough               Basic feature walkthrough\ncustom_objective                Customize loss function, and evaluation metric\nboost_from_prediction           Boosting from existing prediction\npredict_first_ntree             Predicting using first n trees\ngeneralized_linear_model        Generalized Linear Model\ncross_validation                Cross validation\ncreate_sparse_matrix            Create Sparse Matrix\npredict_leaf_indices            Predicting the corresponding leaves\nearly_stopping                  Early Stop in training\npoisson_regression              Poisson regression on count data\ntweedie_regression              Tweedie regression\ngpu_accelerated                 GPU-accelerated tree building algorithms\ninteraction_constraints         Interaction constraints among features'

In [38]:
from langchain.prompts import PromptTemplate

template = """
You are an expert at cleaning and formatting documents.
Write a summary of the document, before re-writing it in cleaner format. Do dot write extra information, while re-writing.

### Original Document :
{text}
###
"""

doc_quality_improving_prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [39]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [40]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
# repo_id = 'dunzhang/stella_en_1.5B_v5'

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    max_new_tokens=15000,
    do_sample=False,
    repetition_penalty=1.03,
    temperature = 0.6
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [41]:
chain_improve_quality = doc_quality_improving_prompt | llm

In [42]:
for i in range(5):
  print("")
  print("<== Original  ==>")
  print(Documents[i].page_content)
  print()
  print("<== Improved ==>")
  print(chain_improve_quality.invoke({'text':Documents[i].page_content}))
  print("-"*100)


<== Original  ==>


basic_walkthrough               Basic feature walkthrough
custom_objective                Customize loss function, and evaluation metric
boost_from_prediction           Boosting from existing prediction
predict_first_ntree             Predicting using first n trees
generalized_linear_model        Generalized Linear Model
cross_validation                Cross validation
create_sparse_matrix            Create Sparse Matrix
predict_leaf_indices            Predicting the corresponding leaves
early_stopping                  Early Stop in training
poisson_regression              Poisson regression on count data
tweedie_regression              Tweedie regression
gpu_accelerated                 GPU-accelerated tree building algorithms
interaction_constraints         Interaction constraints among features

<== Improved ==>

### Summary:
This document outlines various features of the machine learning model, including customizing the loss function and evaluation metric, boost

In [43]:
# prompt: create a function and use try, ecxeption Documents2 = [chain_improve_quality.invoke({'text':x.page_content}) for x in Documents if len(x.page_content)>200]

def improve_docs(docs):
  Documents2 = []
  for x in docs:
    if len(x.page_content)>200:
      try:
        improved_doc = chain_improve_quality.invoke({'text':x.page_content})
        Documents2.append(Document(page_content=improved_doc, metadata=x.metadata))
      except Exception as e:
        print(f"Error processing document: {e}")
  return Documents2

Documents2 = improve_docs(Documents)


Error processing document: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2 (Request ID: EicOoFYHF2lLaVBAWXJi-)

Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32768. Given: 183665 `inputs` tokens and 15000 `max_new_tokens`
Make sure 'text-generation' task is supported by the model.
Error processing document: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2 (Request ID: NwzHyc3xNrI-uciFB5TG1)

Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32768. Given: 742311 `inputs` tokens and 15000 `max_new_tokens`
Make sure 'text-generation' task is supported by the model.
Error processing document: (ReadTimeoutError("HTTPSConnectionPool(host='api-inference.huggingface.co', port=443): Read timed out. (read timeout=120)"), '(Request ID: a383cc68-228a-48dd-97c8-5a6a54f55629)')
Error processing documen

In [44]:
import pickle

In [45]:
with open("Course 5 - SemanticSpotter /Final_Project/Documents2.pkl", "wb") as file:
  pickle.dump(Documents2, file)

In [46]:
with open("Course 5 - SemanticSpotter /Final_Project/Documents2.pkl", "rb") as file:
  x=pickle.load(file)

In [47]:
len(x)

289

## Embedding

In [48]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [49]:
embedding = hf.embed_query(Documents[3].page_content)
len(embedding)

1024

## Saving in VectorDB

In [50]:
from langchain_chroma import Chroma

In [51]:
os.listdir('Course 5 - SemanticSpotter /Final_Project')

['all_files.pkl', '.ipynb_checkpoints', 'chroma_db', 'Documents2.pkl']

In [52]:
# import shutil
# shutil.rmtree('Course 5 - SemanticSpotter /Final_Project/chroma_db')

In [53]:
# os.rmdir('Course 5 - SemanticSpotter /Final_Project/chroma_db')

In [54]:
update_chroma_db = True

In [55]:
if update_chroma_db :
  db = Chroma.from_documents(
      Documents2,
      embedding_function=hf,
      persist_directory="Course 5 - SemanticSpotter /Final_Project/chroma_db_new"
  )
else :
  # load from disk
  db = Chroma(
      persist_directory="Course 5 - SemanticSpotter /Final_Project/chroma_db_new",
      embedding_function=hf
  )

TypeError: langchain_chroma.vectorstores.Chroma() got multiple values for keyword argument 'embedding_function'

In [ ]:
query = "What is the prameter boosters?"

In [ ]:
docs = db.similarity_search(query,k=3)

In [ ]:
pretty_print_docs(docs)

## Retrivers

In [ ]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
# repo_id = 'dunzhang/stella_en_1.5B_v5'

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    temperature = 0.8
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
llm.invoke("Howdy?")

" I'm a software developer based in Berlin, Germany.\n\nI have been developing software for more than 20 years now, mostly in the area of CAD and CAE applications for various industries. In my most recent project, I was working as a CTO at CamptoCamp, where I was leading the development of a cloud-based 3D modelling application for the outdoor industry. We recently finished the beta phase and have just launched our application to the public. I am currently taking a break from the daily work life and am trying out some new projects.\n\nI enjoy learning about new technologies, programming paradigms and algorithms. In my spare time, I like to spend some time on my hobby projects. I've also written some articles on the web and have presented at conferences.\n\nI've developed software for various platforms and languages, among them C++, Java, JavaScript, Python, PHP, Ruby, Rust and Go. I am always happy to learn new things and to explore new technologies.\n\nIf you have any questions or wan

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What is the parameter booster do in XGBOOST?"

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

In [ ]:
retriever_from_llm

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7944880b0910>), llm_chain=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', repetition_penalty=1.03, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>

In [ ]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
retrived_docs = retriever_from_llm.invoke(question)
len(retrived_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['Version 1:', 'Can you explain the role of the booster parameter in XGBoost?', 'Version 2:', "How does XGBoost's booster algorithm function, and what impact does it have on the model?", 'Version 3:', "In XGBoost, what is the significance of the booster component, and how does it contribute to the model's performance?"]


15

In [ ]:
pretty_print_docs(retrived_docs)

Document 1:



0	cap-shape=bell	i
1	cap-shape=conical	i
2	cap-shape=convex	i
3	cap-shape=flat	i
4	cap-shape=knobbed	i
5	cap-shape=sunken	i
6	cap-surface=fibrous	i
7	cap-surface=grooves	i
8	cap-surface=scaly	i
9	cap-surface=smooth	i
10	cap-color=brown	i
11	cap-color=buff	i
12	cap-color=cinnamon	i
13	cap-color=gray	i
14	cap-color=green	i
15	cap-color=pink	i
16	cap-color=purple	i
17	cap-color=red	i
18	cap-color=white	i
19	cap-color=yellow	i
20	bruises?=bruises	i
21	bruises?=no	i
22	odor=almond	i
23	odor=anise	i
24	odor=creosote	i
25	odor=fishy	i
26	odor=foul	i
27	odor=musty	i
28	odor=none	i
29	odor=pungent	i
30	odor=spicy	i
31	gill-attachment=attached	i
32	gill-attachment=descending	i
33	gill-attachment=free	i
34	gill-attachment=notched	i
35	gill-spacing=close	i
36	gill-spacing=crowded	i
37	gill-spacing=distant	i
38	gill-size=broad	i
39	gill-size=narrow	i
40	gill-color=black	i
41	gill-color=brown	i
42	gill-color=buff	i
43	gill-color=chocolate	i
44	gill-color=gray	i
45	gill-color=green	i
4


## Re-Ranking

In [ ]:
!pip install --upgrade --quiet flashrank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

compressor = FlashrankRerank()

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_from_llm
)

compressed_docs = compression_retriever.invoke(
    "What is the parameter booster do in XGBOOST?"
)
# print([doc.metadata["id"] for doc in compressed_docs])

INFO:langchain.retrievers.multi_query:Generated queries: ['Question 1:', 'Can you explain the role of the booster algorithm in XGBOOST?', 'Question 2:', 'In XGBOOST, what function does the booster perform during the training process?', 'Question 3:', "How does XGBOOST's booster differ from other gradient boosting methods and what is its specific contribution?"]


In [ ]:
pretty_print_docs(compressed_docs)

Document 1:

 - INTRODUCTION

['**XGBoost** is an implementation of the famous gradient boosting algorithm. This model is often described as a *blackbox*, meaning it works well but it is not trivial to understand how. Indeed, the model is made of hundreds (thousands?) of decision trees. You may wonder how possible a human would be able to have a general view of the model?\n\nWhile XGBoost is known for its fast speed and accurate predictive power, it also comes with various functions to help you understand the model.\nThe purpose of this RMarkdown document is to demonstrate how easily we can leverage the functions already implemented in **XGBoost R** package. Of course, everything showed below can be applied to the dataset you may have to manipulate at work or wherever!\n\nFirst we will prepare the **Otto** dataset and train a model, then we will generate two visualisations to get a clue of what is important to the model, finally, we will see how we can leverage these information.']
Met

## Generation Layer

In [ ]:
def get_docs(docs):
  return f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nSource: {d.metadata['source']}"
                for i, d in enumerate(docs)
            ]
          )


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
# repo_id = 'dunzhang/stella_en_1.5B_v5'

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    temperature = 0.8
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
###
You love to explain technical concepts, given you have approriate context.
Use the following pieces of context to answer the question given to you.
If the answer is not there in the context, just say that you don't know the answer, don't try to make up an answer.
Your answer must write the references to the source of context the otheriwse it wil be considered as plagarism. The refence link will be provided in the in the metadata.

### Context :
{context}

### Question:
{question}

### Decription of a good answer :
* A good answer will provide reference like provided in academic papers and websites
* A good answer will not contain information which is not provided in the context
* An excellent answer will provide examples with codes to answer the question if it can be provided
"""

gen_promt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [ ]:
chain = gen_promt|llm

In [ ]:
chain.invoke({'context':'idk','question':"what is boosters parameter in xgboost? "})

"\n### Answer: \nThe boosters parameter in XGBoost refers to the type of boosting algorithm used during the training process. XGBoost supports various boosting algorithms such as Gradient Boosting Decision Tree (GBDT), Gradient Boosting Regressor (GBR), and Multi:Softmax for multiclass objective functions. By specifying the booster type, we can choose the appropriate algorithm for our specific problem. For instance, GBDT is suitable for regression problems, while GBR and Multi:Softmax are better options for classification problems.\n\nReference:\n- Chen, T., Meng, Q., Ma, W., Wang, T., Zhu, Y., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System. Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, 1130-1141.\n- https://xgboost.readthedocs.io/en/stable/parameters.html#booster\n\n### Example:\n```python\nimport xgboost as xgb\nX_train, y_train = load_data() # assuming load_data function loads data\n\n# Using Gradient Boosting De

In [ ]:
chain2 = compression_retriever|get_docs

In [ ]:
print(
    chain2.invoke('what is boosters parameter in xgboost?')
)

INFO:langchain.retrievers.multi_query:Generated queries: ['Version 1:', 'Can you explain the function and significance of the booster parameter when using XGBoost?', 'Version 2:', 'How would I set or modify the booster type for an XGBoost model?', 'Version 3:', 'Could you provide some insights into the impact of various booster algorithms in XGBoost and how to select the best one for a specific dataset?']


Document 1:

XGBOOST PARAMETERS - PARAMETERS FOR CATEGORICAL FEATURE - GENERAL PARAMETERS

['Before running XGBoost, we must set three types of parameters: general parameters, booster parameters and task parameters.\n\n- **General parameters** relate to which booster we are using to do boosting, commonly tree or linear model\n- **Booster parameters** depend on which booster you have chosen\n- **Learning task parameters** decide on the learning scenario. For example, regression tasks may use different parameters with ranking tasks.\n- **Command line parameters** relate to behavior of CLI version of XGBoost.\n\n.. note:: Parameters in R package\n\n  In R-package, you can use ``.`` (dot) to replace underscore in the parameters, for example, you can use ``max.depth`` to indicate ``max_depth``. The underscore parameters are also valid in R.\n\n.. contents::\n  :backlinks: none\n  :local:\n\n\n.. _global_config:\n\n********************']
* ``skip_drop``:['[default=0.0]\n\n  - Probability of 

In [ ]:
RAG_chain = (
    {
    "context": retriever_from_llm|get_docs,
    "question": RunnablePassthrough()
    }
    |chain
)

In [ ]:
print(RAG_chain.invoke('what is boosters parameter in xgboost?'))

INFO:langchain.retrievers.multi_query:Generated queries: ['Version 1:', 'Can you explain the function and significance of the booster parameter when using XGBoost?', 'Version 2:', 'How would I set or modify the booster type for an XGBoost model?', 'Version 3:', 'Could you provide some insights into the impact of various booster algorithms in XGBoost and how to select the best one for a specific dataset?']


* An excellent answer will provide explanation of the concept related to the question if it can be provided

### Answer:
Boosters are models used in the XGBoost library for making predictions. The main boosters used in XGBoost are 'gbtree' and 'gblinear'. 'gbtree' is used for classification and regression problems while 'gblinear' is used for regression problems only. Other boosters like 'dart' are also available in XGBoost, which are variations of the standard tree boosters and provide better performance in certain situations.

Booster parameters are specific to the chosen booster and control the way the model is trained and the resulting model's behavior. For example, for the 'gbtree' booster, some important parameters include 'max_depth', 'learning_rate', and 'num_rounds'. These parameters control the maximum depth of the tree, the step size shrinkage used to prevent overfitting, and the number of boosting rounds respectively.

Reference:
* <https://github.com/dmlc/xgboost/blob/mast

### CONTD


In [ ]:
import time
import random
while True :
  time.sleep(random.randint(100,500))

NameError: name 'time' is not defined